In [508]:
import re
# import openpyxl
# from openpyxl.styles import Alignment
from datetime import datetime, timedelta
import pandas as pd

In [509]:
# 날짜 범위 생성
# 현재 날짜 가져오기
today = datetime.today()

# 날짜에 하루를 더하기
tomorrow = today

print(tomorrow)

date_rng = pd.date_range(start='2024-02-19', end=tomorrow, freq='D')

# 데이터프레임 생성
설정맵 = pd.DataFrame(columns=['박연수', '권세한', '권하은',
                            '박지은', '박하은', '손하진',
                            '송호석', '이신후', '이준수', '임혁규'], index=date_rng)


벌금맵 = 설정맵

설정맵 = 설정맵.fillna("설정X")
벌금맵 = 벌금맵.fillna(0)

설정맵.loc['2024-02-19'] = '성공'

for idx, row in 설정맵.iterrows():
    if idx.weekday() == 6:  # 일요일인 경우
        설정맵.loc[idx] = 'off'
        
설정맵

2024-02-28 15:21:04.694159


,박연수,권세한,권하은,박지은,박하은,손하진,송호석,이신후,이준수,임혁규
2024-02-19,성공,성공,성공,성공,성공,성공,성공,성공,성공,성공
2024-02-20,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X
2024-02-21,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X
2024-02-22,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X
2024-02-23,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X
2024-02-24,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X
2024-02-25,off,off,off,off,off,off,off,off,off,off
2024-02-26,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X
2024-02-27,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X
2024-02-28,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X,설정X


In [510]:
file = pd.read_csv("6기.csv", encoding='UTF8')

pattern_set = r'\d{4}\s?\w+\s?설정' # 설정 정규식
pattern_success = r'\d{4}\s?\w+\s?성공' # 성공 정규식
pattern_fail = r'\d{4}\s?\w+\s?실패' # 실패 정규식
pattern_off = r'\d{4}\s?\w+\s?.*?\s?off' # off 정규식
pattern_offs = r'\d{4}-\d{4}\s?\w+\s?.*?\s?off' # offs 정규식
pattern_retry_set = r'\d{4}\s?\w+\s?\d{4}\s?재도전' # 재도전 설정 정규식
pattern_retry_sucess = r'\d{4}\s?\w+\s?\d{4}\s?재성공' # 재도전 인증 정규식

filter_option = f"{pattern_set}|{pattern_success}|{pattern_fail}|{pattern_off}|{pattern_offs}|{pattern_retry_set}|{pattern_retry_sucess}"
filtered_rows = file[file['Message'].str.contains(filter_option, regex=True)]
filtered_rows = filtered_rows.reset_index(drop=True, inplace=False)
filtered_rows['Late'] = 'O'

len_of_rows = filtered_rows.shape[0]
# filtered_rows

In [511]:
# 지각 여부 확인
i = 0
for index in range(len_of_rows):
    DATE = filtered_rows['Date'][index]
    USER = filtered_rows['User'][index]
    MESSAGE = filtered_rows['Message'][index]
    LATE = filtered_rows['Late'][index]

    result_set = str(re.findall(pattern_set, MESSAGE))[2:][:-2]
    result_retry_set = str(re.findall(pattern_retry_set, MESSAGE))[2:][:-2]

    result_success = str(re.findall(pattern_success, MESSAGE))[2:][:-2]
    result_retry_success = str(re.findall(pattern_retry_sucess, MESSAGE))[2:][:-2]
    result_fail = str(re.findall(pattern_fail, MESSAGE))[2:][:-2]
    
    if USER in ['권세한', '임혁규', '이신후', '이준수']:
        DEADLINE = 21
    else :
        DEADLINE = 10
        
    time_str = DATE
    
    time_obj = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')

    
    if len(result_success) >= 1 | len(result_fail) >= 1 | len(result_retry_success) >= 1:
        if  len(result_retry_success) > 1:
            date_str_next = '2024' + result_retry_success[:4]
            
        elif len(result_fail) > 1:
            date_str_next = '2024' + result_fail[:4]
        
        else:
            date_str_next = '2024' + result_success[:4]

        date_obj_next = datetime.strptime(date_str_next + f'{DEADLINE}', "%Y%m%d%H") + timedelta(days=1)
        formatted_date_next = date_obj_next.strftime("%Y-%m-%d %H:%M:%S")
        time_difference_next = time_obj - datetime.strptime(formatted_date_next, "%Y-%m-%d %H:%M:%S")    
        date_str_next = date_obj_next.strftime("%Y-%m-%d")
        date_index_next = datetime.strptime(date_str, '%Y-%m-%d')
    
        if time_difference_next.days > 0 or (time_difference_next.days == 0 and time_difference_next.seconds >= 60):
            filtered_rows['Late'][index] = '지각'
        
    
    elif len(result_set) >= 1 | len(result_retry_set) >= 1:
        if len(result_retry_set) > 1:
            date_str = '2024' + result_retry_set[:4]
        else:
            date_str = '2024' + result_set[:4]

        date_obj = datetime.strptime(date_str + f'{DEADLINE}', "%Y%m%d%H")
        formatted_date = date_obj.strftime("%Y-%m-%d %H:%M:%S")
        time_difference = time_obj - datetime.strptime(formatted_date, "%Y-%m-%d %H:%M:%S")
        date_str = date_obj.strftime("%Y-%m-%d")
        date_index = datetime.strptime(date_str, '%Y-%m-%d')

        if time_difference.days > 0 or (time_difference.days == 0 and time_difference.seconds >= 60):
            filtered_rows['Late'][index] = '지각'

In [512]:
filtered_rows

,Date,User,Message,Late
0,2024-02-19 00:40:35,박지은,0219-0220 박지은 여행으로 off,O
1,2024-02-19 03:19:04,권하은,0129-0220 권하은 새터 및 학생회 사업으로 off,O
2,2024-02-19 03:23:12,권세한,0219 권세한 복귀이슈로 off,O
3,2024-02-19 05:36:24,이준수,0219-0224 휴가로 off,O
4,2024-02-19 13:15:49,이준수,0219-0224 이준수 휴가로 off,O
...,...,...,...,...
89,2024-02-27 23:16:54,송호석,0227 송호석 성공\n0228 송호석 그림체 연구,O
90,2024-02-28 08:49:05,박하은,0227 박하은 실패\n0228 박하은 설정\n- rc 7 듣기\n- 알고리즘 1개,O
91,2024-02-28 08:49:28,박지은,0227 박지은 성공\n0228 박지은 설정\n- 토익 2개 듣기\n- 필라테스\n...,O
92,2024-02-28 09:39:58,권하은,0227 권하은 성공\n0228 권하은 설정\n- 책20쪽읽기\n- 일기쓰기\n- ...,O


In [513]:
#설정 코드
for index in range(len_of_rows):
    DATE = filtered_rows['Date'][index]
    USER = filtered_rows['User'][index]
    MESSAGE = filtered_rows['Message'][index]
    LATE = filtered_rows['Late'][index]

    result_set = str(re.findall(pattern_set, MESSAGE))[2:][:-2]
    result_retry_set = str(re.findall(pattern_retry_set, MESSAGE))[2:][:-2]


    if USER in ['권세한', '임혁규', '이신후', '이준수']:
        DEADLINE = 21
    else :
        DEADLINE = 10

    if len(result_set) <= 1 | len(result_retry_set) <= 1:
        continue
    elif len(result_retry_set) > 1:
        date_str = '2024' + result_retry_set[:4]
    else:
        date_str = '2024' + result_set[:4]

    date_obj = datetime.strptime(date_str + f'{DEADLINE}', "%Y%m%d%H")

    time_str = DATE
    time_obj = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')

    formatted_date = date_obj.strftime("%Y-%m-%d %H:%M:%S")

    time_difference = time_obj - datetime.strptime(formatted_date, "%Y-%m-%d %H:%M:%S")

    # 결과 확인
    date_str = date_obj.strftime("%Y-%m-%d")

    date_index = datetime.strptime(date_str, '%Y-%m-%d')
    
    if filtered_rows['Late'][index] == '지각':
        설정맵.loc[date_index, USER] = '지각'
    else: 
        설정맵.loc[date_index, USER] = 'O'

In [514]:
# 성공 코드
for index in range(len_of_rows):
    DATE = filtered_rows['Date'][index]
    USER = filtered_rows['User'][index]
    MESSAGE = filtered_rows['Message'][index]
    LATE = filtered_rows['Late'][index]

    result_success = str(re.findall(pattern_success, MESSAGE))[2:][:-2]
    result_retry_success = str(re.findall(pattern_retry_sucess, MESSAGE))[2:][:-2]
    result_fail = str(re.findall(pattern_fail, MESSAGE))[2:][:-2]

    if len(result_success) <= 1 | len(result_fail) <= 1 | len(result_retry_success) <= 1:
        continue
    
    if  len(result_retry_success) > 1:
        re.search(pattern_retry_sucess, result_retry_success)
        date_range_str = re.search(r'\d{4}\s?\w+\s?\d{4}', result_retry_success).group()
        date_str = datetime.strptime('2024'+date_range_str[:4], "%Y%m%d")
        target_date = datetime.strptime('2024'+date_range_str[-4:], "%Y%m%d")
        

        if 설정맵.loc[date_str, USER] == 'O':
            설정맵.loc[date_str, USER] = '성공'
            설정맵.loc[target_date, USER] = '재성공'
            continue

        elif 설정맵.loc[date_str, USER] == '지각':
            설정맵.loc[date_str, USER] = '성공'
            설정맵.loc[target_date, USER] = '지각+재성공'
            continue
        
    elif len(result_fail) >= 1:
        date_str = '2024' + result_fail[:4]
    
    else:
        date_str = '2024' + result_success[:4]
        
    date_obj = datetime.strptime(date_str, "%Y%m%d")

    date_str = date_obj.strftime("%Y-%m-%d")

    if filtered_rows['Late'][index]=='지각':
        if len(result_success)  > 1:
            설정맵.loc[date_str, USER] = '지각+성공'
            continue
        
        elif len(result_fail)  > 1: 
            설정맵.loc[date_str, USER] = '지각+실패'
            continue
    
    elif 설정맵.loc[date_str, USER] == 'O':
        if len(result_success)  > 1:
            설정맵.loc[date_str, USER] = '성공'
            continue
        
        elif len(result_fail)  > 1: 
            설정맵.loc[date_str, USER] = '실패'
            continue
        
    elif 설정맵.loc[date_str, USER] == '지각':
        if len(result_success)  > 1:
            설정맵.loc[date_str, USER] = '지각+성공'
            continue
        
        elif len(result_fail)  > 1: 
            설정맵.loc[date_str, USER] = '지각+실패'
            continue

In [515]:
# off 코드
for index in range(len_of_rows):
    DATE = filtered_rows['Date'][index]
    USER = filtered_rows['User'][index]
    MESSAGE = filtered_rows['Message'][index]

    result_off = str(re.findall(pattern_off, MESSAGE))[2:][:-2]
    result_offs = str(re.findall(pattern_offs, MESSAGE))[2:][:-2]

    if len(result_off) <= 1:
        continue
    else:
        date_str = '2024' + result_off[:4]
        
    date_obj = datetime.strptime(date_str, "%Y%m%d")

    date_str = date_obj.strftime("%Y-%m-%d")
    
    if result_off[-3:] == 'off':
        설정맵.loc[date_str, USER] = 'off'
        
    if re.search(pattern_offs, result_offs):
        date_range_str = re.search(r'\d{4}-\d{4}', result_offs).group()
        start_date = datetime.strptime('2024'+date_range_str[:4], "%Y%m%d")
        end_date = datetime.strptime('2024'+date_range_str[5:9], "%Y%m%d") + timedelta(days=1)
        
        # 해당 기간 동안의 행렬을 'off'로 설정
        설정맵.loc[start_date:end_date, USER] = 'off'

In [516]:
# 5일 연속 성공 여부 확인
def check_consecutive_success(user_data):
    consecutive_count = 0
    for status in user_data:
        if status == '성공':
            consecutive_count += 1
            if consecutive_count == 5:
                return True
        else:
            consecutive_count = 0
    return False

# 부여 까방권
for user in 설정맵.columns:
    user_data = 설정맵[user].dropna()  # NaN 값 제외
    if check_consecutive_success(user_data):
        print(f'{user}님에게 까방권 부여!')

In [517]:
설정맵

,박연수,권세한,권하은,박지은,박하은,손하진,송호석,이신후,이준수,임혁규
2024-02-19,성공,off,off,off,성공,성공,성공,성공,off,성공
2024-02-20,off,실패,off,off,성공,실패,성공,off,off,지각+성공
2024-02-21,성공,성공,off,off,성공,성공,성공,지각+성공,off,지각+성공
2024-02-22,성공,성공,off,실패,off,지각+성공,실패,성공,off,off
2024-02-23,성공,off,성공,off,실패,실패,off,실패,off,off
2024-02-24,성공,실패,지각+실패,실패,실패,O,off,off,off,off
2024-02-25,off,off,지각+성공,off,off,off,off,off,off,off
2024-02-26,off,성공,성공,성공,성공,지각+실패,성공,off,성공,off
2024-02-27,off,off,성공,성공,실패,off,성공,off,O,off
2024-02-28,O,off,O,O,O,O,설정X,off,설정X,off


In [518]:
max_consecutive_failures = 3
max_consecutive_successes = 3
max_consecutive_late_failures = 3
max_consecutive_late_sucesses = 3
# 설정맵을 순회하면서 벌금맵에 적절한 값 설정
for person in 설정맵.columns:
    FAIL_STATE = 3000
    LATE_STATE = 1000

    # 연속으로 나온 실패 및 성공 횟수 초기화
    consecutive_failures = 0
    consecutive_successes = 0
    consecutive_late_successes = 0
    consecutive_late_failures = 0

    for date, status in 설정맵[person].items():
        if pd.notna(status):  # NaN이 아닌 경우에만 처리
            time_str = str(date)
            time_obj = datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')

            formatted_date = date_obj.strftime("%Y-%m-%d %H:%M:%S")

            time_difference = time_obj - datetime.today()
            
            if '지각' in str(status):  # '지각'이 문자열에 포함되어 있는지 확인
                벌금맵[person][date] += LATE_STATE
                consecutive_late_failures += 1
                consecutive_late_successes = 0

            else : 
                consecutive_late_successes += 1
                consecutive_late_failures = 0
                
            if '성공' in str(status):
                벌금맵[person][date] += 0
                consecutive_failures = 0
                consecutive_successes += 1
                
                
            elif '실패' in str(status):
                벌금맵[person][date] += FAIL_STATE
                consecutive_failures += 1
                consecutive_successes = 0
    
                
            elif status == 'off':
                벌금맵[person][date] = 0

            elif status == '재성공':
                벌금맵[person][date] = 0
                consecutive_failures = 0
                consecutive_successes += 1

            elif status == '설정X' and  time_difference.days < -2:
                벌금맵[person][date] = FAIL_STATE+LATE_STATE
                consecutive_late_failures += 1
                consecutive_failures += 1
                consecutive_successes = 0
    
            elif status == 'O' and  time_difference.days < -2 :    
                벌금맵[person][date] = FAIL_STATE+LATE_STATE
                consecutive_late_failures += 1
                consecutive_failures += 1
                consecutive_successes = 0
    
            # 연속으로 나온 실패 및 성공 횟수가 설정값 이상이면 벌금 증가 또는 감소
            if consecutive_failures >= max_consecutive_failures:
                FAIL_STATE = 5000
                
            elif consecutive_successes >= max_consecutive_successes:
                FAIL_STATE = 3000

            if consecutive_late_failures >= max_consecutive_late_failures:
                LATE_STATE = 2000
                
            elif consecutive_late_successes >= max_consecutive_late_sucesses:
                LATE_STATE = 1000
                    
                


In [519]:
벌금맵

,박연수,권세한,권하은,박지은,박하은,손하진,송호석,이신후,이준수,임혁규
2024-02-19,0,0,0,0,0,0,0,0,0,0
2024-02-20,0,3000,0,0,0,3000,0,0,0,1000
2024-02-21,0,0,0,0,0,0,0,1000,0,1000
2024-02-22,0,0,0,3000,0,1000,3000,0,0,0
2024-02-23,0,0,0,0,3000,3000,0,3000,0,0
2024-02-24,0,3000,4000,3000,3000,4000,0,0,0,0
2024-02-25,0,0,1000,0,0,0,0,0,0,0
2024-02-26,0,0,0,0,0,4000,0,0,0,0
2024-02-27,0,0,0,0,3000,0,0,0,0,0
2024-02-28,0,0,0,0,0,0,0,0,0,0


In [520]:
# datetime 인덱스를 'YYYY-MM-DD' 포맷으로 변경
벌금맵.index = 벌금맵.index.map(lambda x: x.strftime('%Y-%m-%d') if isinstance(x, pd.Timestamp) else x)
설정맵.index = 설정맵.index.map(lambda x: x.strftime('%Y-%m-%d') if isinstance(x, pd.Timestamp) else x)

# 각 열의 합을 계산하여 마지막 행에 추가
벌금맵.loc['합계'] = 벌금맵.sum()

# 출력
print(벌금맵)


            박연수   권세한   권하은   박지은   박하은    손하진   송호석   이신후  이준수   임혁규
2024-02-19    0     0     0     0     0      0     0     0    0     0
2024-02-20    0  3000     0     0     0   3000     0     0    0  1000
2024-02-21    0     0     0     0     0      0     0  1000    0  1000
2024-02-22    0     0     0  3000     0   1000  3000     0    0     0
2024-02-23    0     0     0     0  3000   3000     0  3000    0     0
2024-02-24    0  3000  4000  3000  3000   4000     0     0    0     0
2024-02-25    0     0  1000     0     0      0     0     0    0     0
2024-02-26    0     0     0     0     0   4000     0     0    0     0
2024-02-27    0     0     0     0  3000      0     0     0    0     0
2024-02-28    0     0     0     0     0      0     0     0    0     0
합계            0  6000  5000  6000  9000  15000  3000  4000    0  2000


In [521]:
설정맵

,박연수,권세한,권하은,박지은,박하은,손하진,송호석,이신후,이준수,임혁규
2024-02-19,성공,off,off,off,성공,성공,성공,성공,off,성공
2024-02-20,off,실패,off,off,성공,실패,성공,off,off,지각+성공
2024-02-21,성공,성공,off,off,성공,성공,성공,지각+성공,off,지각+성공
2024-02-22,성공,성공,off,실패,off,지각+성공,실패,성공,off,off
2024-02-23,성공,off,성공,off,실패,실패,off,실패,off,off
2024-02-24,성공,실패,지각+실패,실패,실패,O,off,off,off,off
2024-02-25,off,off,지각+성공,off,off,off,off,off,off,off
2024-02-26,off,성공,성공,성공,성공,지각+실패,성공,off,성공,off
2024-02-27,off,off,성공,성공,실패,off,성공,off,O,off
2024-02-28,O,off,O,O,O,O,설정X,off,설정X,off


In [523]:
맵 = pd.concat((벌금맵, 설정맵))

맵.to_csv('mycsv.csv')